# **1**

# 1.1: A Baseline Neural Netowrk Tagger

In [23]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torch.nn as nn
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np

In [24]:
def parse_file(file):
    """
    Parse the TSV file for the POS tagging task.
    Args:
    file (file object): The file object to be read.
    Returns:
    list: A list of sentences, where each sentence is represented as a list of (word, tag) tuples.
    """
    sentences = []
    sentence = []
    for line in file:
        line = line.strip()
        if line:
            word, pos = line.split('\t')
            sentence.append((word, pos))
        else:
            sentences.append(sentence)
            sentence = []

    if sentence:
        sentences.append(sentence)
    return sentences

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

def load_data(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        sentences = parse_file(file)
    return sentences

In [26]:
train_sentences = load_data("twpos-train.tsv")
dev_sentences = load_data("twpos-dev.tsv")
devtest_sentences = load_data("twpos-devtest.tsv")

In [27]:
def prepare_data():
    train_data = train_sentences

    word_to_ix = {"UUUNKKK": 0, '<s>':1, '</s>':2}
    tag_to_ix = {}

    # Iterate over each sentence (which is a list of (word, tag) tuples)
    for sentence in train_data:
        for word, tag in sentence:
            if word not in word_to_ix:
                # Add the word to the dictionary if it's not already there
                word_to_ix[word] = len(word_to_ix)

            if tag not in tag_to_ix:
                tag_to_ix[tag] = len(tag_to_ix)
    return word_to_ix, tag_to_ix

# Usage
word_to_ix, tag_to_ix = prepare_data()
target_size = len(tag_to_ix)

In [28]:
def transfer_sentence(data, word_to_ix, tag_to_ix, w):
  concat_data = []
  tags = []
  for sentence in data:
      for i in range(len(sentence)):
          current = []
          for j in range(i - w, i + w + 1):
              if j < 0:
                word = '</s>'
              elif j >= len(sentence):
                word = '</s>'
              else:
                word = sentence[j][0]
              if word not in word_to_ix:
                word = 'UUUNKKK'
              current.append(word_to_ix[word])
              if i == j:
                tag = tag_to_ix[sentence[j][1]]
          concat_data.append(current)
          tags.append(tag)
  return np.array(concat_data), np.array(tags)

In [29]:
class POSTagger:
  def __init__(self, w, vocab_size):
    self.w = w
    self.dim_e = 50
    self.dim_h = 128
    self.dim_s = len(tag_to_ix)
    self.vocab_size = len(word_to_ix)
    self.batch_size = 32
    self.epochs = 20

In [30]:
class NN(nn.Module):
    def __init__(self, POSTagger):
        super().__init__()
        self.POSTagger = POSTagger
        self.embeddings = nn.Embedding(self.POSTagger.vocab_size, self.POSTagger.dim_e)
        self.hidden = nn.Linear((2 * self.POSTagger.w + 1) * self.POSTagger.dim_e, self.POSTagger.dim_h)
        self.output = nn.Linear(self.POSTagger.dim_h, self.POSTagger.dim_s)

        self.loss_function = F.cross_entropy
        self.optimizer = optim.SGD(self.parameters(), lr=0.02)

        self.init_weights()

    def init_weights(self):
        initrange = 0.01
        self.embeddings.weight.data.uniform_(-initrange, initrange)
        self.hidden.weight.data.uniform_(-initrange, initrange)
        self.output.weight.data.uniform_(-initrange, initrange)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view(-1, (2 * self.POSTagger.w + 1) * self.POSTagger.dim_e)
        #print(embeds.shape)
        hidden_out = self.hidden(embeds)
        hidden_activated = torch.tanh(hidden_out)
        tag_space = self.output(hidden_activated)
        tag_scores = F.log_softmax(tag_space, dim=-1)
        return tag_scores

    def load_data(self, train_sentence, dev_sentence, devtest_sentence, word_to_ix, tag_to_ix):
        self.train_data, self.train_label = transfer_sentence(train_sentence, word_to_ix, tag_to_ix, self.POSTagger.w)
        self.dev_data, self.dev_label = transfer_sentence(dev_sentence, word_to_ix, tag_to_ix, self.POSTagger.w)
        self.devtest_data, self.devtest_label = transfer_sentence(devtest_sentence, word_to_ix, tag_to_ix, self.POSTagger.w)

    def get_loss(self, x, y):
        log_prob = self.forward(x)
        loss = self.loss_function(log_prob, y, reduction='sum')
        return loss

    def run_grad(self, x, y):
        loss = self.get_loss(x, y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss

    def one_epoch(self, epoch, sentence, label):
        n = sentence.shape[0]
        idx = np.arange(0, n)
        np.random.shuffle(idx)

        sentence_s = sentence[idx]
        label_s = label[idx]

        train_loss = 0
        for i in range(0, n, self.POSTagger.batch_size):
            x = torch.tensor(sentence_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            y = torch.tensor(label_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            loss = self.run_grad(x, y)
            train_loss += loss.item()


        train_loss /= n
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}')
        return train_loss

    def test(self, sentence, label):
        self.eval()
        n = sentence.shape[0]
        correct = 0

        with torch.no_grad():
            test_loss = 0
            x = torch.tensor(sentence, dtype=torch.long)
            y = torch.tensor(label, dtype=torch.long)
            loss = self.get_loss(x, y)
            test_loss += loss.item()

            log_probs = self.forward(x)
            _, predicted = torch.max(log_probs.data, 1)
            correct += (y == predicted).sum().item()

            test_loss /= n
            accuracy = (correct / n) * 100
            print(f"Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}%")
            return test_loss

    def fit(self):
        best_dev_loss = np.inf
        epochs_without_improvement = 0
        patience = 5

        for i in range(self.POSTagger.epochs):
            train_loss = self.one_epoch(i, self.train_data, self.train_label)
            dev_loss = self.test(self.dev_data, self.dev_label)

            if dev_loss < best_dev_loss:
                best_dev_loss = dev_loss
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= patience:
                    print("Early stopping due to no improvement.")
                    break
        print('\nTesting Result')
        devtest_loss = self.test(self.devtest_data, self.devtest_label)
        return train_loss, dev_loss, devtest_loss

The experiment demonstrated a clear performance improvement when the context window size was increased from w=0 to w=1, highlighting the model's enhanced ability to leverage additional contextual information for more accurate predictions. This suggests that a larger context window is crucial for capturing necessary context and nuances in language processing tasks.

In [31]:
if __name__ == "__main__":
    window_sizes = [0, 1]

    for w in window_sizes:
        print(f"\nTraining with context window size w={w}")
        POST = POSTagger(w=w, vocab_size = len(word_to_ix))
        model = NN(POST)
        model.load_data(train_sentences, dev_sentences, devtest_sentences, word_to_ix, tag_to_ix)
        model.fit()


Training with context window size w=0
Epoch 0, Loss: 2.5900
Loss: 2.0071, Accuracy: 34.4534%
Epoch 1, Loss: 1.6070
Loss: 1.2447, Accuracy: 67.9942%
Epoch 2, Loss: 1.0065
Loss: 1.0844, Accuracy: 74.0510%
Epoch 3, Loss: 0.7084
Loss: 1.0854, Accuracy: 75.4200%
Epoch 4, Loss: 0.5326
Loss: 1.1063, Accuracy: 76.2497%
Epoch 5, Loss: 0.4253
Loss: 1.1199, Accuracy: 77.5565%
Epoch 6, Loss: 0.3792
Loss: 1.0919, Accuracy: 77.2869%
Epoch 7, Loss: 0.3429
Loss: 1.1398, Accuracy: 77.1417%
Early stopping due to no improvement.

Testing Result
Loss: 1.0484, Accuracy: 78.1850%

Training with context window size w=1
Epoch 0, Loss: 2.4949
Loss: 1.6656, Accuracy: 46.5671%
Epoch 1, Loss: 1.0897
Loss: 0.9314, Accuracy: 73.1176%
Epoch 2, Loss: 0.6643
Loss: 0.8416, Accuracy: 78.7181%
Epoch 3, Loss: 0.4425
Loss: 0.8401, Accuracy: 80.2531%
Epoch 4, Loss: 0.3368
Loss: 0.8810, Accuracy: 79.0500%
Epoch 5, Loss: 0.2599
Loss: 0.9111, Accuracy: 79.9834%
Epoch 6, Loss: 0.2009
Loss: 0.9401, Accuracy: 79.7760%
Epoch 7, L

# 1.2 Feature Engineering

Upon reviewing misclassified center words from section 1.1, it was observed that inaccuracies often involved words beginning with capital letters, containing special characters, or including numbers. To address this, new features were designed to precisely identify such traits within the words. Incorporating these nuanced features enhanced the model's understanding of the data, leading to a noticeable improvement in accuracy compared to the results in section 1.1.

In [32]:
import string
def extract_features(word):
    features = [
        float(any(char.isdigit() for char in word)),  # contains digits
        float(word.isdigit()),
        float(any(char in string.punctuation for char in word)),  # contains punctuation
        word[0] in string.punctuation,
        len(word)
    ]
    return features

In [33]:
def transfer_sentence_features(sentences, w):
  feature_matrix = []
  for sentence in sentences:
      for i in range(len(sentence)):
          for j in range(i - w, i + w + 1):
                if i == j:
                  features = extract_features(sentence[j][0])
                  feature_matrix.append(features)
  return np.array(feature_matrix)

In [34]:
class NN_feature(nn.Module):
    def __init__(self, POSTagger):
        super().__init__()
        self.POSTagger = POSTagger
        self.num_features = 5
        self.embeddings = nn.Embedding(self.POSTagger.vocab_size, self.POSTagger.dim_e)
        self.hidden = nn.Linear((2 * self.POSTagger.w + 1) * self.POSTagger.dim_e  + self.num_features, self.POSTagger.dim_h)
        self.output = nn.Linear(self.POSTagger.dim_h, self.POSTagger.dim_s)
        self.loss_function = F.cross_entropy
        self.optimizer = optim.SGD(self.parameters(), lr=0.02)
        self.init_weights()

    def init_weights(self):
        initrange = 0.01
        self.embeddings.weight.data.uniform_(-initrange, initrange)
        self.hidden.weight.data.uniform_(-initrange, initrange)
        self.output.weight.data.uniform_(-initrange, initrange)

    def forward(self, inputs):
        inputs, features = inputs
        embeds = self.embeddings(inputs).view(-1, ((2 * self.POSTagger.w + 1) * self.POSTagger.dim_e))
        combined = torch.cat((embeds, features), 1)
        hidden_out = self.hidden(combined)
        hidden_activated = torch.tanh(hidden_out)
        tag_space = self.output(hidden_activated)
        tag_scores = F.log_softmax(tag_space, dim=-1)
        return tag_scores

    def load_data(self, train_sentence, dev_sentence, devtest_sentence, word_to_ix, tag_to_ix):
        self.train_data, self.train_label = transfer_sentence(train_sentence, word_to_ix, tag_to_ix, self.POSTagger.w)
        self.train_features = transfer_sentence_features(train_sentence, self.POSTagger.w)
        self.dev_data, self.dev_label = transfer_sentence(dev_sentence, word_to_ix, tag_to_ix, self.POSTagger.w)
        self.dev_features = transfer_sentence_features(dev_sentence, self.POSTagger.w)

        self.devtest_data, self.devtest_label = transfer_sentence(devtest_sentence, word_to_ix, tag_to_ix, self.POSTagger.w)
        self.devtest_features = transfer_sentence_features(devtest_sentence, self.POSTagger.w)

        # Convert the feature data from NumPy arrays to tensors
        self.train_features = torch.tensor(self.train_features, dtype=torch.float)
        self.dev_features = torch.tensor(self.dev_features, dtype=torch.float)
        self.devtest_features = torch.tensor(self.devtest_features, dtype=torch.float)

    def get_loss(self, x, y):
        log_prob = self.forward(x)
        loss = self.loss_function(log_prob, y, reduction='sum')
        return loss

    def run_grad(self, x, y):
        loss = self.get_loss(x, y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss

    def one_epoch(self, epoch, sentence, label, features):
        n = sentence.shape[0]
        idx = np.arange(0, n)
        np.random.shuffle(idx)

        sentence_s = sentence[idx]
        label_s = label[idx]
        feature_s = features[idx]
        train_loss = 0
        for i in range(0, n, self.POSTagger.batch_size):
            x = torch.tensor(sentence_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            y = torch.tensor(label_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            feature_batch = feature_s[i:i + self.POSTagger.batch_size].clone().detach()

            loss = self.run_grad((x, feature_batch), y)
            train_loss += loss.item()

        train_loss /= n
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}')
        return train_loss

    def test(self, sentence, label, features):
        self.eval()
        n = sentence.shape[0]
        correct = 0

        with torch.no_grad():
            test_loss = 0
            x = torch.tensor(sentence, dtype=torch.long)
            y = torch.tensor(label, dtype=torch.long)
            feature_batch = features.clone().detach()
            loss = self.get_loss((x, feature_batch), y)
            test_loss += loss.item()

            log_probs = self.forward((x,feature_batch))
            _, predicted = torch.max(log_probs.data, 1)
            correct += (y == predicted).sum().item()

        test_loss /= n
        accuracy = (correct / n) * 100
        print(f"Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}%")
        return test_loss

    def fit(self):
        best_dev_loss = np.inf
        epochs_without_improvement = 0
        patience = 5

        for i in range(self.POSTagger.epochs):
            train_loss = self.one_epoch(i, self.train_data, self.train_label, self.train_features)
            dev_loss = self.test(self.dev_data, self.dev_label, self.dev_features)

            if dev_loss < best_dev_loss:
                best_dev_loss = dev_loss
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= patience:
                    print("Early stopping due to no improvement.")
                    break
        print('\nTesting Result')
        devtest_loss = self.test(self.devtest_data, self.devtest_label, self.devtest_features)
        return train_loss, dev_loss, devtest_loss

In [35]:
if __name__ == "__main__":
    window_sizes = [0, 1]

    for w in window_sizes:
        print(f"\nTraining with context window size w={w}")
        POST = POSTagger(w=w, vocab_size = len(word_to_ix))
        model = NN_feature(POST)
        model.load_data(train_sentences, dev_sentences, devtest_sentences, word_to_ix, tag_to_ix)
        model.fit()


Training with context window size w=0
Epoch 0, Loss: 1.9213
Loss: 1.3358, Accuracy: 56.5235%
Epoch 1, Loss: 1.1825
Loss: 1.1400, Accuracy: 66.8326%
Epoch 2, Loss: 0.9413
Loss: 0.9109, Accuracy: 73.7399%
Epoch 3, Loss: 0.7751
Loss: 1.0900, Accuracy: 73.4287%
Epoch 4, Loss: 0.6167
Loss: 0.9947, Accuracy: 75.2126%
Epoch 5, Loss: 0.5438
Loss: 1.0348, Accuracy: 75.5445%
Epoch 6, Loss: 0.4647
Loss: 1.0432, Accuracy: 77.5150%
Epoch 7, Loss: 0.4046
Loss: 1.0194, Accuracy: 77.2039%
Early stopping due to no improvement.

Testing Result
Loss: 0.9623, Accuracy: 78.6808%

Training with context window size w=1
Epoch 0, Loss: 1.9358
Loss: 1.3448, Accuracy: 57.7681%
Epoch 1, Loss: 1.1161
Loss: 0.9713, Accuracy: 68.9691%
Epoch 2, Loss: 0.8139
Loss: 0.7943, Accuracy: 75.2748%
Epoch 3, Loss: 0.6369
Loss: 0.7428, Accuracy: 79.5271%
Epoch 4, Loss: 0.5153
Loss: 0.7614, Accuracy: 78.2410%
Epoch 5, Loss: 0.3979
Loss: 0.7333, Accuracy: 80.3568%
Epoch 6, Loss: 0.3226
Loss: 0.6962, Accuracy: 81.6013%
Epoch 7, L

# 1.3 Pretrained Embeddings

1) Experiment with updating the pretrained embeddings for w = 0 and w = 1

* Setting up the train an embedding for '&lt;s&gt;' by using the embedding for '&lt;s&gt;'

In [36]:
import pandas as pd
def load_embeddings(embedding_file):
    embeddings = {}
    with open(embedding_file, 'r', encoding='utf-8') as f:
      lines = f.readlines()
      for line in lines:
        words = line.split()
        embeddings[words[0]] = words[1:]
        if words[0] == '<\s>':
          embeddings['<s>'] = words[1:]
    return pd.DataFrame.from_dict(embeddings, orient = 'index').astype('float32')

In [37]:
embedding_df = load_embeddings('/content/twitter-embeddings.txt')

In [38]:
twitter_to_index = {word: i for i, word in enumerate(embedding_df.index)}

In [39]:
class POSTagger_embed:
  def __init__(self, w, vocab_size):
    self.w = w
    self.dim_e = 50
    self.dim_h = 128
    self.dim_s = len(tag_to_ix)
    self.vocab_size = len(twitter_to_index)
    self.batch_size = 32
    self.epochs = 20

In [40]:
class NN_embed(nn.Module):
    def __init__(self, POSTagger, pertrained_embeddings, twitter_to_index):
        super().__init__()
        self.POSTagger = POSTagger
        self.embeddings = nn.Embedding(self.POSTagger.vocab_size, self.POSTagger.dim_e)
        self.hidden = nn.Linear((2 * self.POSTagger.w + 1) * self.POSTagger.dim_e, self.POSTagger.dim_h)
        self.output = nn.Linear(self.POSTagger.dim_h, self.POSTagger.dim_s)

        self.loss_function = F.cross_entropy
        self.optimizer = optim.SGD(self.parameters(), lr=0.02)

        # Initialize embeddings with pretrained vectors
        self.init_embeddings(pretrained_embeddings, twitter_to_index)

    def init_embeddings(self, pretrained_embeddings, twitter_to_index):
        initrange = 0.01
        self.embeddings.weight.data = pretrained_embeddings
        self.hidden.weight.data.uniform_(-initrange, initrange)
        self.output.weight.data.uniform_(-initrange, initrange)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view(-1, (2 * self.POSTagger.w + 1) * self.POSTagger.dim_e)
        #print(embeds.shape)
        hidden_out = self.hidden(embeds)
        hidden_activated = torch.tanh(hidden_out)
        tag_space = self.output(hidden_activated)
        tag_scores = F.log_softmax(tag_space, dim=-1)
        return tag_scores

    def load_data(self, train_sentence, dev_sentence, devtest_sentence, twitter_to_index, tag_to_ix):
        self.train_data, self.train_label = transfer_sentence(train_sentence, twitter_to_index, tag_to_ix, self.POSTagger.w)
        self.dev_data, self.dev_label = transfer_sentence(dev_sentence, twitter_to_index, tag_to_ix, self.POSTagger.w)
        self.devtest_data, self.devtest_label = transfer_sentence(devtest_sentence, twitter_to_index, tag_to_ix, self.POSTagger.w)

    def get_loss(self, x, y):
        log_prob = self.forward(x)
        loss = self.loss_function(log_prob, y, reduction='sum')
        return loss

    def run_grad(self, x, y):
        loss = self.get_loss(x, y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss

    def one_epoch(self, epoch, sentence, label):
        n = sentence.shape[0]
        idx = np.arange(0, n)
        np.random.shuffle(idx)

        sentence_s = sentence[idx]
        label_s = label[idx]

        train_loss = 0
        for i in range(0, n, self.POSTagger.batch_size):
            x = torch.tensor(sentence_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            y = torch.tensor(label_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            loss = self.run_grad(x, y)
            train_loss += loss.item()
        train_loss /= n
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}')
        return train_loss

    def test(self, sentence, label):
        self.eval()
        n = sentence.shape[0]
        correct = 0

        with torch.no_grad():
            test_loss = 0
            x = torch.tensor(sentence, dtype=torch.long)
            y = torch.tensor(label, dtype=torch.long)
            loss = self.get_loss(x, y)
            test_loss += loss.item()

            log_probs = self.forward(x)
            _, predicted = torch.max(log_probs.data, 1)
            correct += (y == predicted).sum().item()

            test_loss /= n
            accuracy = (correct / n) * 100
            print(f"Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}%")
            return test_loss, accuracy

    def fit(self):
        best_dev_loss = np.inf
        epochs_without_improvement = 0
        patience = 5

        for i in range(self.POSTagger.epochs):
            print('Epoch', i)
            train_loss = self.one_epoch(i, self.train_data, self.train_label)
            dev_loss, dev_accuracy = self.test(self.dev_data, self.dev_label)

            if dev_loss < best_dev_loss:
                best_dev_loss = dev_loss
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= patience:
                    print("Early stopping due to no improvement.")
                    break
            print('------------------------------------')
        print('\nTesting Result')
        devtest_loss, devtest_accuracy = self.test(self.devtest_data, self.devtest_label)
        return train_loss, dev_loss, devtest_loss, dev_accuracy, devtest_accuracy

The experiment demonstrated accuracy improvements for both w=0 and w=1 settings when using pre-trained embeddings, as opposed to randomly-initialized word embeddings. This underscores the superiority of adopting pre-trained embeddings for enhanced model performance.

In [41]:
if __name__ == "__main__":
    window_sizes = [0, 1]
    pretrained_embeddings = torch.tensor(embedding_df.values)
    for w in window_sizes:
        print(f"\nTraining with context window size w={w}")
        POST = POSTagger_embed(w=w, vocab_size = len(embedding_df))
        model = NN_embed(POST, pretrained_embeddings, twitter_to_index)
        model.load_data(train_sentences, dev_sentences, devtest_sentences, twitter_to_index, tag_to_ix)
        model.fit()


Training with context window size w=0
Epoch 0
Epoch 0, Loss: 1.0320
Loss: 0.6079, Accuracy: 83.4267%
------------------------------------
Epoch 1
Epoch 1, Loss: 0.5181
Loss: 0.6042, Accuracy: 82.9496%
------------------------------------
Epoch 2
Epoch 2, Loss: 0.4424
Loss: 0.5458, Accuracy: 83.4474%
------------------------------------
Epoch 3
Epoch 3, Loss: 0.4035
Loss: 0.5605, Accuracy: 82.4310%
------------------------------------
Epoch 4
Epoch 4, Loss: 0.3849
Loss: 0.5669, Accuracy: 83.8000%
------------------------------------
Epoch 5
Epoch 5, Loss: 0.3709
Loss: 0.5730, Accuracy: 83.1363%
------------------------------------
Epoch 6
Epoch 6, Loss: 0.3612
Loss: 0.5700, Accuracy: 82.6799%
------------------------------------
Epoch 7
Epoch 7, Loss: 0.3563
Loss: 0.5835, Accuracy: 82.9081%
Early stopping due to no improvement.

Testing Result
Loss: 0.5359, Accuracy: 83.3369%

Training with context window size w=1
Epoch 0
Epoch 0, Loss: 0.5975
Loss: 0.5199, Accuracy: 85.4802%
---------

2) Comparing the result for updating the pretrained word embeddings during training and keeping them fixed

In [42]:
class NN_embed_freeze(nn.Module):
    def __init__(self, POSTagger, pretrained_embeddings, twitter_to_index, freeze_embeddings):
        super().__init__()
        self.POSTagger = POSTagger
        self.freeze_embeddings = freeze_embeddings
        self.embeddings = nn.Embedding(self.POSTagger.vocab_size, self.POSTagger.dim_e)
        self.hidden = nn.Linear((2 * self.POSTagger.w + 1) * self.POSTagger.dim_e, self.POSTagger.dim_h)
        self.output = nn.Linear(self.POSTagger.dim_h, self.POSTagger.dim_s)

        self.loss_function = F.cross_entropy
        self.optimizer = optim.SGD(filter(lambda p: p.requires_grad, self.parameters()), lr=0.02)

        # Initialize embeddings with pretrained vectors
        self.init_embeddings(pretrained_embeddings)

    def init_embeddings(self, pretrained_embeddings):
        initrange = 0.01
        self.embeddings.weight.data = pretrained_embeddings
        self.embeddings.weight.requires_grad = not self.freeze_embeddings
        self.hidden.weight.data.uniform_(-initrange, initrange)
        self.output.weight.data.uniform_(-initrange, initrange)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view(-1, (2 * self.POSTagger.w + 1) * self.POSTagger.dim_e)

        hidden_out = self.hidden(embeds)
        hidden_activated = torch.tanh(hidden_out)
        tag_space = self.output(hidden_activated)
        tag_scores = F.log_softmax(tag_space, dim=-1)
        return tag_scores

    def load_data(self, train_sentence, dev_sentence, devtest_sentence, twitter_to_index, tag_to_ix):
        self.train_data, self.train_label = transfer_sentence(train_sentence, twitter_to_index, tag_to_ix, self.POSTagger.w)
        self.dev_data, self.dev_label = transfer_sentence(dev_sentence, twitter_to_index, tag_to_ix, self.POSTagger.w)
        self.devtest_data, self.devtest_label = transfer_sentence(devtest_sentence, twitter_to_index, tag_to_ix, self.POSTagger.w)

    def get_loss(self, x, y):
        log_prob = self.forward(x)
        loss = self.loss_function(log_prob, y, reduction='sum')
        return loss

    def run_grad(self, x, y):
        loss = self.get_loss(x, y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss

    def one_epoch(self, epoch, sentence, label):
        n = sentence.shape[0]
        idx = np.arange(0, n)
        np.random.shuffle(idx)

        sentence_s = sentence[idx]
        label_s = label[idx]

        train_loss = 0
        for i in range(0, n, self.POSTagger.batch_size):
            x = torch.tensor(sentence_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            y = torch.tensor(label_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            loss = self.run_grad(x, y)
            train_loss += loss.item()
        train_loss /= n
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}')
        return train_loss

    def test(self, sentence, label):
        self.eval()
        n = sentence.shape[0]
        correct = 0

        with torch.no_grad():
            test_loss = 0
            x = torch.tensor(sentence, dtype=torch.long)
            y = torch.tensor(label, dtype=torch.long)
            loss = self.get_loss(x, y)
            test_loss += loss.item()

            log_probs = self.forward(x)
            _, predicted = torch.max(log_probs.data, 1)
            correct += (y == predicted).sum().item()

            test_loss /= n
            accuracy = (correct / n) * 100
            print(f"Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}%")
            return test_loss

    def fit(self):
        best_dev_loss = np.inf
        epochs_without_improvement = 0
        patience = 5

        for i in range(self.POSTagger.epochs):
            print('Epoch', i)
            train_loss = self.one_epoch(i, self.train_data, self.train_label)
            dev_loss = self.test(self.dev_data, self.dev_label)

            if dev_loss < best_dev_loss:
                best_dev_loss = dev_loss
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= patience:
                    print("Early stopping due to no improvement.")
                    break

            print('------------------------------------------')

        print('\nTesting Result')
        devtest_loss = self.test(self.devtest_data, self.devtest_label)
        return train_loss, dev_loss, devtest_loss

During training with a context window of w=1, updating pre-trained word embeddings has shown distinct benefits, evidenced by a modest yet noteworthy improvement of approximately 2% in accuracy compared to keeping them fixed. This enhancement underscores the effectiveness of allowing the embeddings to evolve with training.

In [43]:
if __name__ == "__main__":
    freeze = [True, False]
    pretrained_embeddings = torch.tensor(embedding_df.values)
    for f in freeze:
        print(f"\nParameter Freeze = {f}")
        POST = POSTagger_embed(w = 1, vocab_size = len(embedding_df))
        model = NN_embed_freeze(POST, pretrained_embeddings, twitter_to_index, f)
        model.load_data(train_sentences, dev_sentences, devtest_sentences, twitter_to_index, tag_to_ix)
        model.fit()


Parameter Freeze = True
Epoch 0
Epoch 0, Loss: 1.0161
Loss: 0.6291, Accuracy: 82.4310%
------------------------------------------
Epoch 1
Epoch 1, Loss: 0.5568
Loss: 0.5863, Accuracy: 83.6963%
------------------------------------------
Epoch 2
Epoch 2, Loss: 0.5014
Loss: 0.5332, Accuracy: 85.2935%
------------------------------------------
Epoch 3
Epoch 3, Loss: 0.4665
Loss: 0.5152, Accuracy: 85.2935%
------------------------------------------
Epoch 4
Epoch 4, Loss: 0.4430
Loss: 0.5113, Accuracy: 85.4594%
------------------------------------------
Epoch 5
Epoch 5, Loss: 0.4219
Loss: 0.5103, Accuracy: 85.8121%
------------------------------------------
Epoch 6
Epoch 6, Loss: 0.4047
Loss: 0.5055, Accuracy: 85.5424%
------------------------------------------
Epoch 7
Epoch 7, Loss: 0.3877
Loss: 0.5037, Accuracy: 86.0195%
------------------------------------------
Epoch 8
Epoch 8, Loss: 0.3727
Loss: 0.4963, Accuracy: 86.3099%
------------------------------------------
Epoch 9
Epoch 9, Loss

3) Feature Engineering for pretrained embeddings model

In [44]:
class NN_embed_feature(nn.Module):
    def __init__(self, POSTagger, pertrained_embeddings, twitter_to_index):
        super().__init__()
        self.POSTagger = POSTagger
        self.num_features = 5
        self.embeddings = nn.Embedding(self.POSTagger.vocab_size, self.POSTagger.dim_e)
        self.hidden = nn.Linear((2 * self.POSTagger.w + 1) * self.POSTagger.dim_e  + self.num_features, self.POSTagger.dim_h)
        self.output = nn.Linear(self.POSTagger.dim_h, self.POSTagger.dim_s)
        self.loss_function = F.cross_entropy
        self.optimizer = optim.SGD(self.parameters(), lr=0.02)

        # Initialize embeddings with pretrained vectors
        self.init_embeddings(pretrained_embeddings, twitter_to_index)

    def init_embeddings(self, pretrained_embeddings, twitter_to_index):
        initrange = 0.01
        self.embeddings.weight.data = pretrained_embeddings
        self.hidden.weight.data.uniform_(-initrange, initrange)
        self.output.weight.data.uniform_(-initrange, initrange)
    def forward(self, inputs):
        inputs, features = inputs
        embeds = self.embeddings(inputs).view(-1, ((2 * self.POSTagger.w + 1) * self.POSTagger.dim_e))
        combined = torch.cat((embeds, features), 1)
        hidden_out = self.hidden(combined)
        hidden_activated = torch.tanh(hidden_out)
        tag_space = self.output(hidden_activated)
        tag_scores = F.log_softmax(tag_space, dim=-1)
        return tag_scores

    def load_data(self, train_sentence, dev_sentence, devtest_sentence, word_to_ix, tag_to_ix):
        self.train_data, self.train_label = transfer_sentence(train_sentence, word_to_ix, tag_to_ix, self.POSTagger.w)
        self.train_features = transfer_sentence_features(train_sentence, self.POSTagger.w)
        self.dev_data, self.dev_label = transfer_sentence(dev_sentence, word_to_ix, tag_to_ix, self.POSTagger.w)
        self.dev_features = transfer_sentence_features(dev_sentence, self.POSTagger.w)

        self.devtest_data, self.devtest_label = transfer_sentence(devtest_sentence, word_to_ix, tag_to_ix, self.POSTagger.w)
        self.devtest_features = transfer_sentence_features(devtest_sentence, self.POSTagger.w)

        # Convert the feature data from NumPy arrays to tensors
        self.train_features = torch.tensor(self.train_features, dtype=torch.float)
        self.dev_features = torch.tensor(self.dev_features, dtype=torch.float)
        self.devtest_features = torch.tensor(self.devtest_features, dtype=torch.float)

    def get_loss(self, x, y):
        log_prob = self.forward(x)
        loss = self.loss_function(log_prob, y, reduction='sum')
        return loss

    def run_grad(self, x, y):
        loss = self.get_loss(x, y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss

    def one_epoch(self, epoch, sentence, label, features):
        n = sentence.shape[0]
        idx = np.arange(0, n)
        np.random.shuffle(idx)

        sentence_s = sentence[idx]
        label_s = label[idx]
        feature_s = features[idx]
        train_loss = 0
        for i in range(0, n, self.POSTagger.batch_size):
            x = torch.tensor(sentence_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            y = torch.tensor(label_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            feature_batch = feature_s[i:i + self.POSTagger.batch_size].clone().detach()
            loss = self.run_grad((x, feature_batch), y)
            train_loss += loss.item()

        train_loss /= n
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}')
        return train_loss

    def test(self, sentence, label, features):
        self.eval()
        n = sentence.shape[0]
        correct = 0

        with torch.no_grad():
            test_loss = 0
            x = torch.tensor(sentence, dtype=torch.long)
            y = torch.tensor(label, dtype=torch.long)
            feature_batch = features.clone().detach()
            loss = self.get_loss((x, feature_batch), y)
            test_loss += loss.item()

            log_probs = self.forward((x,feature_batch))
            _, predicted = torch.max(log_probs.data, 1)
            correct += (y == predicted).sum().item()

        test_loss /= n
        accuracy = (correct / n) * 100
        print(f"Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}%")
        return test_loss

    def fit(self):
        best_dev_loss = np.inf
        epochs_without_improvement = 0
        patience = 5

        for i in range(self.POSTagger.epochs):
            print('Epoch', i)
            train_loss = self.one_epoch(i, self.train_data, self.train_label, self.train_features)
            dev_loss = self.test(self.dev_data, self.dev_label, self.dev_features)

            if dev_loss < best_dev_loss:
                best_dev_loss = dev_loss
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= patience:
                    print("Early stopping due to no improvement.")
                    break
            print('------------------------')
        print('\nTesting Result')
        devtest_loss = self.test(self.devtest_data, self.devtest_label, self.devtest_features)
        return train_loss, dev_loss, devtest_loss

Combining the custom features from Section 1.2 with pre-trained embeddings reveals that these features still offer assistance. Although the improvement is subtle, it highlights that features can contribute additional context that pretraining might not capture, thereby providing a slight edge in the model's performance.

In [45]:
if __name__ == "__main__":
    window_sizes = [0, 1]
    pretrained_embeddings = torch.tensor(embedding_df.values)
    for w in window_sizes:
        print(f"\nTraining with context window size w={w}")
        POST = POSTagger_embed(w=w, vocab_size = len(embedding_df))
        model = NN_embed_feature(POST, pretrained_embeddings, twitter_to_index)
        model.load_data(train_sentences, dev_sentences, devtest_sentences, twitter_to_index, tag_to_ix)
        model.fit()


Training with context window size w=0
Epoch 0
Epoch 0, Loss: 1.2374
Loss: 0.6162, Accuracy: 81.2072%
------------------------
Epoch 1
Epoch 1, Loss: 0.6115
Loss: 0.5858, Accuracy: 81.8917%
------------------------
Epoch 2
Epoch 2, Loss: 0.5042
Loss: 0.5449, Accuracy: 84.0075%
------------------------
Epoch 3
Epoch 3, Loss: 0.4308
Loss: 0.6345, Accuracy: 82.8044%
------------------------
Epoch 4
Epoch 4, Loss: 0.3902
Loss: 0.5461, Accuracy: 83.6963%
------------------------
Epoch 5
Epoch 5, Loss: 0.3773
Loss: 0.5465, Accuracy: 83.9867%
------------------------
Epoch 6
Epoch 6, Loss: 0.3592
Loss: 0.5755, Accuracy: 84.4638%
------------------------
Epoch 7
Epoch 7, Loss: 0.3490
Loss: 0.5793, Accuracy: 83.5719%
Early stopping due to no improvement.

Testing Result
Loss: 0.5350, Accuracy: 83.7465%

Training with context window size w=1
Epoch 0
Epoch 0, Loss: 0.7493
Loss: 0.5819, Accuracy: 85.1691%
------------------------
Epoch 1
Epoch 1, Loss: 0.3173
Loss: 0.5568, Accuracy: 85.2313%
-----

# 1.4 Architecture Engineering

**I got the best test accuracy when window size = 1 and no hidden layer with tanh function.**

1) Compare the use of 0, 1, and 2 hidden layers

In [46]:
class NN_layers(nn.Module):
    def __init__(self, POSTagger, pertrained_embeddings, twitter_to_index, hidden_dim1=None, hidden_dim2=None):
        super().__init__()
        self.POSTagger = POSTagger
        self.embeddings = nn.Embedding(self.POSTagger.vocab_size, self.POSTagger.dim_e)
        self.output = nn.Linear(self.POSTagger.dim_h, self.POSTagger.dim_s)
        self.hidden1 = None
        self.hidden2 = None

        input_dim = (2 * self.POSTagger.w + 1) * self.POSTagger.dim_e

        if hidden_dim1:
            self.hidden1 = nn.Linear(input_dim, hidden_dim1)
            output_dim = hidden_dim1

            if hidden_dim2:
                self.hidden2 = nn.Linear(hidden_dim1, hidden_dim2)
                output_dim = hidden_dim2
        else:
            output_dim = input_dim
        self.output = nn.Linear(output_dim, self.POSTagger.dim_s)
        self.loss_function = F.cross_entropy
        self.optimizer = optim.SGD(self.parameters(), lr=0.02)

        # Initialize embeddings with pretrained vectors
        self.init_embeddings(pretrained_embeddings, twitter_to_index)

    def init_embeddings(self, pretrained_embeddings, twitter_to_index):
        initrange = 0.01
        self.embeddings.weight.data = pretrained_embeddings
        if self.hidden1 is not None:
          self.hidden1.weight.data.uniform_(-initrange, initrange)
        if self.hidden2 is not None:
          self.hidden2.weight.data.uniform_(-initrange, initrange)
        self.output.weight.data.uniform_(-initrange, initrange)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view(-1, (2 * self.POSTagger.w + 1) * self.POSTagger.dim_e)
        if self.hidden1 is not None:
          hidden_out = self.hidden1(embeds)
          if self.hidden2 is not None:
            hidden_out = self.hidden2(hidden_out)
        else:
          hidden_out = embeds
        hidden_activated = torch.tanh(hidden_out)
        tag_space = self.output(hidden_activated)
        tag_scores = F.log_softmax(tag_space, dim=-1)
        return tag_scores

    def load_data(self, train_sentence, dev_sentence, devtest_sentence, twitter_to_index, tag_to_ix):
        self.train_data, self.train_label = transfer_sentence(train_sentence, twitter_to_index, tag_to_ix, self.POSTagger.w)
        self.dev_data, self.dev_label = transfer_sentence(dev_sentence, twitter_to_index, tag_to_ix, self.POSTagger.w)
        self.devtest_data, self.devtest_label = transfer_sentence(devtest_sentence, twitter_to_index, tag_to_ix, self.POSTagger.w)

    def get_loss(self, x, y):
        log_prob = self.forward(x)
        loss = self.loss_function(log_prob, y, reduction='sum')
        return loss

    def run_grad(self, x, y):
        loss = self.get_loss(x, y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss

    def one_epoch(self, epoch, sentence, label):
        n = sentence.shape[0]
        idx = np.arange(0, n)
        np.random.shuffle(idx)

        sentence_s = sentence[idx]
        label_s = label[idx]

        train_loss = 0
        for i in range(0, n, self.POSTagger.batch_size):
            x = torch.tensor(sentence_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            y = torch.tensor(label_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            loss = self.run_grad(x, y)
            train_loss += loss.item()

        train_loss /= n
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}')
        return train_loss

    def test(self, sentence, label):
        self.eval()
        n = sentence.shape[0]
        correct = 0

        with torch.no_grad():
            test_loss = 0
            x = torch.tensor(sentence, dtype=torch.long)
            y = torch.tensor(label, dtype=torch.long)
            loss = self.get_loss(x, y)
            test_loss += loss.item()

            log_probs = self.forward(x)
            _, predicted = torch.max(log_probs.data, 1)
            correct += (y == predicted).sum().item()

            test_loss /= n
            accuracy = (correct / n) * 100
            print(f"Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}%")
            return test_loss, accuracy

    def fit(self):
        best_dev_loss = np.inf
        epochs_without_improvement = 0
        patience = 5

        for i in range(self.POSTagger.epochs):
            print('Epoch', i)
            train_loss = self.one_epoch(i, self.train_data, self.train_label)
            dev_loss, dev_accuracy = self.test(self.dev_data, self.dev_label)

            if dev_loss < best_dev_loss:
                best_dev_loss = dev_loss
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= patience:
                    print("Early stopping due to no improvement.")
                    break
            print('---------------------------------------------')
        print('\n Testing Result')
        devtest_loss, devtest_accuracy = self.test(self.devtest_data, self.devtest_label)
        return train_loss, dev_loss, devtest_loss, dev_accuracy, devtest_accuracy

In [47]:
configs = [
    # 0 hidden layers
   {'hidden_layers': 0, 'hidden_dim1': None, 'hidden_dim2': None},

    # 1 hidden layer, two setups
    {'hidden_layers': 1, 'hidden_dim1': 256, 'hidden_dim2': None},
    {'hidden_layers': 1, 'hidden_dim1': 512, 'hidden_dim2': None},

    # 2 hidden layers, two setups
    {'hidden_layers': 2, 'hidden_dim1': 256, 'hidden_dim2': 128},
    {'hidden_layers': 2, 'hidden_dim1': 512, 'hidden_dim2': 256}
]

# Adding window sizes to test
window_sizes = [0, 1]

# This will be used to collect all results
all_results = []

for window in window_sizes:
    results = []  # This collects results for the current window size

    for config in configs:
        print(f"Training configuration: {config}, Window size: {window}")

        # Initialize the model with the current configuration
        pretrained_embeddings = torch.tensor(embedding_df.values)
        POST = POSTagger_embed(w=window, vocab_size=len(embedding_df))
        model = NN_layers(POST, pretrained_embeddings, twitter_to_index,
                          hidden_dim1=config['hidden_dim1'],
                          hidden_dim2=config['hidden_dim2'])
        model.load_data(train_sentences, dev_sentences, devtest_sentences, twitter_to_index, tag_to_ix)

        result = model.fit()

        if isinstance(result, tuple):
            result = {
                'train_loss': result[0],
                'dev_loss': result[1],
                'devtest_loss': result[2],
                'dev_accuracy': result[3],
                'devtest_accuracy': result[4]
            }

        result['window_size'] = window
        result.update(config)
        results.append(result)
    results_df = pd.DataFrame(results)
    print(f"Results for window size {window}:\n", results_df)
    all_results.extend(results)
all_results_df = pd.DataFrame(all_results)
print("Complete results:\n", all_results_df)

Training configuration: {'hidden_layers': 0, 'hidden_dim1': None, 'hidden_dim2': None}, Window size: 0
Epoch 0
Epoch 0, Loss: 0.9941
Loss: 0.6275, Accuracy: 81.2280%
---------------------------------------------
Epoch 1
Epoch 1, Loss: 0.5282
Loss: 0.5521, Accuracy: 83.1363%
---------------------------------------------
Epoch 2
Epoch 2, Loss: 0.4452
Loss: 0.5322, Accuracy: 84.3186%
---------------------------------------------
Epoch 3
Epoch 3, Loss: 0.4007
Loss: 0.5265, Accuracy: 84.2771%
---------------------------------------------
Epoch 4
Epoch 4, Loss: 0.3729
Loss: 0.5343, Accuracy: 83.1155%
---------------------------------------------
Epoch 5
Epoch 5, Loss: 0.3577
Loss: 0.5497, Accuracy: 83.8623%
---------------------------------------------
Epoch 6
Epoch 6, Loss: 0.3465
Loss: 0.5443, Accuracy: 83.8623%
---------------------------------------------
Epoch 7
Epoch 7, Loss: 0.3387
Loss: 0.5386, Accuracy: 84.3601%
---------------------------------------------
Epoch 8
Epoch 8, Loss: 0.

Both for the window sizes 0 and 1, the test accuracy decreased as the model increased the number of layers and neurons in the layers.

In [48]:
all_results_df

,train_loss,dev_loss,devtest_loss,dev_accuracy,devtest_accuracy,window_size,hidden_layers,hidden_dim1,hidden_dim2
0,0.334195,0.551624,0.514400,83.074051,83.703384,0,0,NaN,NaN
1,0.356924,0.573442,0.531290,84.297864,84.479414,0,1,256.0,NaN
2,0.342210,0.582594,0.543496,84.028210,84.177624,0,1,512.0,NaN
3,0.332495,0.689333,0.615828,83.302219,83.617159,0,2,256.0,128.0
4,0.339902,0.666445,0.614963,82.472516,82.366889,0,2,512.0,256.0
5,0.133885,0.469412,0.419780,87.450736,88.445786,1,0,NaN,NaN
6,0.151463,0.625214,0.568134,86.372122,87.152404,1,1,256.0,NaN
7,0.148190,0.628975,0.553454,86.102468,87.152404,1,1,512.0,NaN
8,0.172792,0.678500,0.638390,85.231280,86.182367,1,2,256.0,128.0
9,0.198769,0.701571,0.650674,84.795686,86.096141,1,2,512.0,256.0


2) Experiment with different nonlinearities(Identity, tanh, ReLU, Sigmoid) with number of hidden layers = 1, hidden layer of width = 128

In [57]:
class NN_activation(nn.Module):
    def __init__(self, POSTagger, pertrained_embeddings, twitter_to_index, activation_function):
        super().__init__()
        self.POSTagger = POSTagger
        self.embeddings = nn.Embedding(self.POSTagger.vocab_size, self.POSTagger.dim_e)
        self.hidden = nn.Linear((2 * self.POSTagger.w + 1) * self.POSTagger.dim_e, self.POSTagger.dim_h)
        self.output = nn.Linear(self.POSTagger.dim_h, self.POSTagger.dim_s)

        self.loss_function = F.cross_entropy
        self.optimizer = optim.SGD(self.parameters(), lr=0.02)

        # Initialize embeddings with pretrained vectors
        self.init_embeddings(pretrained_embeddings)

        # Set the activation function
        if activation_function == 'relu':
            self.activation = nn.ReLU()
        elif activation_function == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation_function == 'identity':
            self.activation = lambda x: x
        else:
            self.activation = nn.Tanh()  # Default is tanh

    def init_embeddings(self, pretrained_embeddings):
        initrange = 0.01
        self.embeddings.weight.data = pretrained_embeddings
        self.hidden.weight.data.uniform_(-initrange, initrange)
        self.output.weight.data.uniform_(-initrange, initrange)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view(-1, (2 * self.POSTagger.w + 1) * self.POSTagger.dim_e)
        hidden_out = self.hidden(embeds)
        hidden_activated = self.activation(hidden_out)
        tag_space = self.output(hidden_activated)
        tag_scores = F.log_softmax(tag_space, dim=-1)
        return tag_scores

    def load_data(self, train_sentence, dev_sentence, devtest_sentence, twitter_to_index, tag_to_ix):
        self.train_data, self.train_label = transfer_sentence(train_sentence, twitter_to_index, tag_to_ix, self.POSTagger.w)
        self.dev_data, self.dev_label = transfer_sentence(dev_sentence, twitter_to_index, tag_to_ix, self.POSTagger.w)
        self.devtest_data, self.devtest_label = transfer_sentence(devtest_sentence, twitter_to_index, tag_to_ix, self.POSTagger.w)

    def get_loss(self, x, y):
        log_prob = self.forward(x)
        loss = self.loss_function(log_prob, y, reduction='sum')
        return loss

    def run_grad(self, x, y):
        loss = self.get_loss(x, y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss

    def one_epoch(self, epoch, sentence, label):
        n = sentence.shape[0]
        idx = np.arange(0, n)
        np.random.shuffle(idx)

        sentence_s = sentence[idx]
        label_s = label[idx]

        train_loss = 0
        for i in range(0, n, self.POSTagger.batch_size):
            x = torch.tensor(sentence_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            y = torch.tensor(label_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            loss = self.run_grad(x, y)
            train_loss += loss.item()

        train_loss /= n
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}')
        return train_loss

    def test(self, sentence, label):
        self.eval()
        n = sentence.shape[0]
        correct = 0

        with torch.no_grad():
            test_loss = 0
            x = torch.tensor(sentence, dtype=torch.long)
            y = torch.tensor(label, dtype=torch.long)
            loss = self.get_loss(x, y)
            test_loss += loss.item()

            log_probs = self.forward(x)
            _, predicted = torch.max(log_probs.data, 1)
            correct += (y == predicted).sum().item()

            test_loss /= n
            accuracy = (correct / n) * 100
            print(f"Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}%")
            return test_loss, accuracy

    def fit(self):
        best_dev_loss = np.inf
        epochs_without_improvement = 0
        patience = 5

        for i in range(self.POSTagger.epochs):
            print('Epoch', i)
            train_loss = self.one_epoch(i, self.train_data, self.train_label)
            dev_loss, dev_accuracy = self.test(self.dev_data, self.dev_label)

            if dev_loss < best_dev_loss:
                best_dev_loss = dev_loss
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= patience:
                    print("Early stopping due to no improvement.")
                    break
            print('------------------------------------')
        print('\nTesting Result')
        devtest_loss, devtest_accuracy = self.test(self.devtest_data, self.devtest_label)
        return train_loss, dev_loss, devtest_loss, dev_accuracy, devtest_accuracy

In [50]:
configs = []

# Define configurations based on window sizes and activation functions
window_sizes = [0, 1]
activation_functions = ['identity', 'tanh', 'relu', 'sigmoid']

for w in window_sizes:
    for activation in activation_functions:
        configs.append({
            'window_size': w,
            'activation_function': activation
        })

results = []

for config in configs:
    print(f"\nTraining with context window size w={config['window_size']} and {config['activation_function']} activation function")

    pretrained_embeddings = torch.tensor(embedding_df.values)

    POST = POSTagger_embed(w=config['window_size'], vocab_size=len(embedding_df))
    model = NN_activation(POST, pretrained_embeddings, twitter_to_index, activation_function=config['activation_function'])

    model.load_data(train_sentences, dev_sentences, devtest_sentences, twitter_to_index, tag_to_ix)

    result = model.fit()

    if isinstance(result, tuple):
        result = {
            'train_loss': result[0],
            'dev_loss': result[1],
            'devtest_loss': result[2],
            'dev_accuracy': result[3],
            'devtest_accuracy': result[4]
        }

    result.update(config)

    results.append(result)

results_df = pd.DataFrame(results)

print(results_df)



Training with context window size w=0 and identity activation function
Epoch 0
Epoch 0, Loss: 1.0195
Loss: 0.6811, Accuracy: 81.9125%
------------------------------------
Epoch 1
Epoch 1, Loss: 0.5119
Loss: 0.5864, Accuracy: 82.5347%
------------------------------------
Epoch 2
Epoch 2, Loss: 0.4376
Loss: 0.6198, Accuracy: 82.9496%
------------------------------------
Epoch 3
Epoch 3, Loss: 0.4094
Loss: 0.5471, Accuracy: 84.0282%
------------------------------------
Epoch 4
Epoch 4, Loss: 0.3883
Loss: 0.5787, Accuracy: 83.8623%
------------------------------------
Epoch 5
Epoch 5, Loss: 0.3765
Loss: 0.5977, Accuracy: 82.5347%
------------------------------------
Epoch 6
Epoch 6, Loss: 0.3701
Loss: 0.5595, Accuracy: 84.0904%
------------------------------------
Epoch 7
Epoch 7, Loss: 0.3641
Loss: 0.5660, Accuracy: 83.0948%
------------------------------------
Epoch 8
Epoch 8, Loss: 0.3588
Loss: 0.5555, Accuracy: 82.5970%
Early stopping due to no improvement.

Testing Result
Loss: 0.510

Overly, ReLU or Sigmoid produce the highest test accuracy, but it doesn't differ that much.

In [51]:
results_df

,train_loss,dev_loss,devtest_loss,dev_accuracy,devtest_accuracy,window_size,activation_function
0,0.358820,0.555467,0.510834,82.596972,83.358482,0,identity
1,0.353601,0.561695,0.527724,83.965982,83.940504,0,tanh
2,0.325073,0.571744,0.533019,84.007467,84.005174,0,relu
3,0.326105,0.600742,0.542025,82.908110,83.918948,0,sigmoid
4,0.157767,0.605181,0.555561,85.666874,86.742833,1,identity
5,0.111449,0.637382,0.583445,85.791330,86.721276,1,tanh
6,0.120856,0.594233,0.544088,86.911429,87.432636,1,relu
7,0.160995,0.554030,0.521123,86.849201,87.432636,1,sigmoid


3) Experiment with w = 2 and compare the results to w = 0 and 1

In [52]:
configs = [{'window_size': w} for w in [0, 1, 2]]

results = []

for config in configs:
    print(f"\nTraining with context window size w={config['window_size']}")
    pretrained_embeddings = torch.tensor(embedding_df.values)
    POST = POSTagger_embed(w=config['window_size'], vocab_size=len(embedding_df))
    model = NN_embed(POST, pretrained_embeddings, twitter_to_index)
    model.load_data(train_sentences, dev_sentences, devtest_sentences, twitter_to_index, tag_to_ix)
    result = model.fit()

    if isinstance(result, tuple):
        result = {
            'train_loss': result[0],
            'dev_loss': result[1],
            'devtest_loss': result[2],
            'dev_accuracy': result[3],
            'devtest_accuracy': result[4]
        }

    result.update(config)

    results.append(result)

results_df = pd.DataFrame(results)

results_df



Training with context window size w=0
Epoch 0
Epoch 0, Loss: 1.0315
Loss: 0.5859, Accuracy: 83.3644%
------------------------------------
Epoch 1
Epoch 1, Loss: 0.5158
Loss: 0.6137, Accuracy: 82.5347%
------------------------------------
Epoch 2
Epoch 2, Loss: 0.4422
Loss: 0.6108, Accuracy: 83.3644%
------------------------------------
Epoch 3
Epoch 3, Loss: 0.4095
Loss: 0.5785, Accuracy: 83.4267%
------------------------------------
Epoch 4
Epoch 4, Loss: 0.3921
Loss: 0.6036, Accuracy: 81.4146%
------------------------------------
Epoch 5
Epoch 5, Loss: 0.3782
Loss: 0.5565, Accuracy: 83.8415%
------------------------------------
Epoch 6
Epoch 6, Loss: 0.3681
Loss: 0.5460, Accuracy: 83.8415%
------------------------------------
Epoch 7
Epoch 7, Loss: 0.3606
Loss: 0.5430, Accuracy: 84.3808%
------------------------------------
Epoch 8
Epoch 8, Loss: 0.3580
Loss: 0.5547, Accuracy: 83.3022%
------------------------------------
Epoch 9
Epoch 9, Loss: 0.3491
Loss: 0.6111, Accuracy: 82.1821

,train_loss,dev_loss,devtest_loss,dev_accuracy,devtest_accuracy,window_size
0,0.339709,0.567317,0.513823,83.800041,84.285406,0
1,0.148761,0.584993,0.533822,86.621033,87.432636,1
2,0.086527,0.656453,0.624116,85.998755,87.109291,2


Window size 1 produces the best test accuracy, and we can also see that the bigger window size does not make the test accuracy higher.


In [53]:
results_df

,train_loss,dev_loss,devtest_loss,dev_accuracy,devtest_accuracy,window_size
0,0.339709,0.567317,0.513823,83.800041,84.285406,0
1,0.148761,0.584993,0.533822,86.621033,87.432636,1
2,0.086527,0.656453,0.624116,85.998755,87.109291,2


# 1.5 RNN Taggers

In [54]:
class POSTagger_Additional:
  def __init__(self, w, vocab_size):
    self.w = w
    self.dim_e = 50
    self.dim_h = 512
    self.dim_s = len(tag_to_ix)
    self.vocab_size = len(twitter_to_index)
    self.batch_size = 32
    self.epochs = 20

In [55]:
class NN_Additional(nn.Module):
    def __init__(self, POSTagger, pertrained_embeddings, twitter_to_index, rnn_type, bidirectional):
        super().__init__()
        self.POSTagger = POSTagger
        self.embeddings = nn.Embedding(self.POSTagger.vocab_size, self.POSTagger.dim_e)
        self.hidden = nn.Linear((2 * self.POSTagger.w + 1) * self.POSTagger.dim_e, self.POSTagger.dim_h)
        direction_factor = 2 if bidirectional else 1
        self.output = nn.Linear(self.POSTagger.dim_h * direction_factor, self.POSTagger.dim_s)
        rnn_class = {'RNN': nn.RNN, 'LSTM': nn.LSTM, 'GRU': nn.GRU}[rnn_type]
        self.rnn = rnn_class(self.POSTagger.dim_e * (2 * self.POSTagger.w + 1), self.POSTagger.dim_h, batch_first=True, bidirectional=bidirectional)
        # Output layer

        self.dropout_first = nn.Dropout(0.5)

        self.loss_function = F.cross_entropy
        self.optimizer = optim.SGD(self.parameters(), lr=0.02)

        # Initialize embeddings with pretrained vectors
        self.init_embeddings(pretrained_embeddings, twitter_to_index)

    def init_embeddings(self, pretrained_embeddings, twitter_to_index):
        """
        Initialize the embedding layer with pretrained embeddings.
        Words not found in the pretrained list will be initialized with the UUUNKKK vector.
        """
        initrange = 0.01
        self.embeddings.weight.data = pretrained_embeddings
     #   self.rnn.weight.data.uniform_(-initrange, initrange)
        self.output.weight.data.uniform_(-initrange, initrange)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view(-1, (2 * self.POSTagger.w + 1) * self.POSTagger.dim_e)
        embeds = self.dropout_first(embeds)
        hidden_out = self.rnn(embeds)
        hidden_out, _ = self.rnn(embeds)
        hidden_activated = F.relu(hidden_out)
        tag_space = self.output(hidden_activated)
        tag_scores = F.log_softmax(tag_space, dim=-1)
        return tag_scores

    def load_data(self, train_sentence, dev_sentence, devtest_sentence, twitter_to_index, tag_to_ix):
        self.train_data, self.train_label = transfer_sentence(train_sentence, twitter_to_index, tag_to_ix, self.POSTagger.w)
        self.dev_data, self.dev_label = transfer_sentence(dev_sentence, twitter_to_index, tag_to_ix, self.POSTagger.w)
        self.devtest_data, self.devtest_label = transfer_sentence(devtest_sentence, twitter_to_index, tag_to_ix, self.POSTagger.w)

    def get_loss(self, x, y):
        log_prob = self.forward(x)
        loss = self.loss_function(log_prob, y, reduction='sum')
        return loss

    def run_grad(self, x, y):
        loss = self.get_loss(x, y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss

    def one_epoch(self, epoch, sentence, label):
        n = sentence.shape[0]
        idx = np.arange(0, n)
        np.random.shuffle(idx)

        sentence_s = sentence[idx]
        label_s = label[idx]

        train_loss = 0
        for i in range(0, n, self.POSTagger.batch_size):
            x = torch.tensor(sentence_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            y = torch.tensor(label_s[i:i + self.POSTagger.batch_size], dtype=torch.long)
            loss = self.run_grad(x, y)
            train_loss += loss.item()
        train_loss /= n
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}')
        return train_loss

    def test(self, sentence, label):
        self.eval()
        n = sentence.shape[0]
        correct = 0

        with torch.no_grad():
            test_loss = 0
            x = torch.tensor(sentence, dtype=torch.long)
            y = torch.tensor(label, dtype=torch.long)
            loss = self.get_loss(x, y)
            test_loss += loss.item()

            log_probs = self.forward(x)
            _, predicted = torch.max(log_probs.data, 1)
            correct += (y == predicted).sum().item()

            test_loss /= n
            accuracy = (correct / n) * 100
            print(f"Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}%")
            return test_loss, accuracy

    def fit(self):
        best_dev_loss = np.inf
        epochs_without_improvement = 0
        patience = 5

        for i in range(self.POSTagger.epochs):
            print('Epoch', i)
            train_loss = self.one_epoch(i, self.train_data, self.train_label)
            dev_loss, dev_accuracy = self.test(self.dev_data, self.dev_label)

            if dev_loss < best_dev_loss:
                best_dev_loss = dev_loss
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= patience:
                    print("Early stopping due to no improvement.")
                    break
            print('------------------------------------')
        print('\nTesting Result')
        devtest_loss, devtest_accuracy = self.test(self.devtest_data, self.devtest_label)
        return train_loss, dev_loss, devtest_loss, dev_accuracy, devtest_accuracy

In [56]:
POST = POSTagger_Additional(w=2, vocab_size = len(embedding_df))
def train_and_evaluate_models(configurations):
    results = []

    for config in configurations:
        rnn_type, bidirectional = config['rnn_type'], config['bidirectional']
        print(f"\nTraining model with RNN type: {rnn_type} | Bidirectional: {str(bidirectional)}")

        model = NN_Additional(POST, pretrained_embeddings, twitter_to_index, rnn_type=rnn_type, bidirectional=bidirectional)

        model.load_data(train_sentences, dev_sentences, devtest_sentences, twitter_to_index, tag_to_ix)

        train_loss, dev_loss, devtest_loss, dev_accuracy, devtest_accuracy = model.fit()

        results.append({
            'rnn_type': rnn_type,
            'bidirectional': bidirectional,
            'train_loss': train_loss,
            'dev_loss': dev_loss,
            'devtest_loss': devtest_loss,
            'dev_accuracy': dev_accuracy,
            'devtest_accuracy': devtest_accuracy
        })

    return pd.DataFrame(results)

configurations = [
    {'rnn_type': 'LSTM', 'bidirectional': True},
    {'rnn_type': 'RNN', 'bidirectional': True},
    {'rnn_type': 'GRU', 'bidirectional': True},
    {'rnn_type': 'LSTM', 'bidirectional': False},
    {'rnn_type': 'RNN', 'bidirectional': False},
    {'rnn_type': 'GRU', 'bidirectional': False}
]

results_df = train_and_evaluate_models(configurations)

results_df


Training model with RNN type: LSTM | Bidirectional: True
Epoch 0
Epoch 0, Loss: 1.2639
Loss: 0.5565, Accuracy: 83.8830%
------------------------------------
Epoch 1
Epoch 1, Loss: 0.2705
Loss: 0.5150, Accuracy: 86.5173%
------------------------------------
Epoch 2
Epoch 2, Loss: 0.1618
Loss: 0.5580, Accuracy: 86.4136%
------------------------------------
Epoch 3
Epoch 3, Loss: 0.1147
Loss: 0.5770, Accuracy: 86.4758%
------------------------------------
Epoch 4
Epoch 4, Loss: 0.0847
Loss: 0.6352, Accuracy: 86.0610%
------------------------------------
Epoch 5
Epoch 5, Loss: 0.0655
Loss: 0.6614, Accuracy: 85.9365%
------------------------------------
Epoch 6
Epoch 6, Loss: 0.0553
Loss: 0.7078, Accuracy: 85.7084%
Early stopping due to no improvement.

Testing Result
Loss: 0.6527, Accuracy: 86.3333%

Training model with RNN type: RNN | Bidirectional: True
Epoch 0
Epoch 0, Loss: 7.7915
Loss: 6.1588, Accuracy: 28.7077%
------------------------------------
Epoch 1
Epoch 1, Loss: 4.1154
Loss:

,rnn_type,bidirectional,train_loss,dev_loss,devtest_loss,dev_accuracy,devtest_accuracy
0,LSTM,True,0.055341,0.707769,0.652717,85.708359,86.333261
1,RNN,True,2.382645,1.833507,1.796937,71.831570,72.084501
2,GRU,True,0.084065,0.792066,0.736939,84.277121,85.406338
3,LSTM,False,0.075654,0.728211,0.644470,84.235636,85.751239
4,RNN,False,6.365451,8.030508,7.952078,30.014520,31.558526
5,GRU,False,0.386780,1.075713,1.005192,77.266127,78.120285
